# Test file for existing models

After Loading Emotion, Pain, SR Models. Testing on given samples occurs to prove the degree of accuracy of the models on external pictures that weren't used in training, validation or Main.ipynb testing

### Change in Directory

Changing to parent directory, just for the sake of easening the far away and nested directories file imports

In [ ]:
import os
os.chdir('..')
print os.getcwd() # Home directory of the whole project --> py.pain-detection/

### Imports 

For libraries used and Configuration file for variables

In [ ]:
from keras.models import load_model
from keras.preprocessing import image as image_utils
import numpy as np
import matplotlib.pyplot as plt
import main.utils.config as conf

#### Redirected to original directory for consistency

In [ ]:
os.chdir('test/')

### Loading the models for testing

In [ ]:
path = '../main/'
test_imgs_path = 'samples/'
test_pain_imgs_path = 'pain_samples/'
emotions_model = load_model(path+'model_emo_adam_50ep_Inception_plateau.h5')
pain_model_adam = load_model(path+'model_pain_adam_240ep_2ndtrial_plateau.h5')
pain_model_rmsprop = load_model(path+'model_pain_rmsprop_240ep_3rdtrial_plateau.h5')
pain_model_datagen = load_model(path+'model_pain_nogcloud_datagen_240ep.h5')
sr_model = load_model(path+'model_finalSR_3densemodel_CrossVal.h5')

## PNG/JPG/JPEG parse and Emotion detection and Pain measurement

#### - Function for loading images to required shape matrices

In [ ]:
def imgToColoredMatrix(fileName):
    image = image_utils.load_img(fileName, target_size=(conf.PIC_DIM_PAIN, conf.PIC_DIM_PAIN))
    image = image_utils.img_to_array(image).astype(np.float32)
    image = image/ 255.
    
    print
    print "colored image ", fileName, " :"
    
    plt.imshow(image)
    plt.show()
    
    return image

#### - Emotion Prediction showing scores for each emotion and predicting pain level if pain was detected

In [ ]:
for f in os.listdir(test_imgs_path):
    fileName = os.path.join(test_imgs_path, f)
    
    image = imgToColoredMatrix(fileName)
    
    # img to gray scale matrix
    gray_image = image_utils.load_img(fileName, target_size=(conf.PICTURE_DIM_EMOTION, conf.PICTURE_DIM_EMOTION))
    gray_image = image_utils.img_to_array(gray_image).astype(np.float32)
    gray_image = gray_image/ 255.
    gray_image = np.dot(gray_image[..., :3], [0.299, 0.587, 0.114])
    print "gray image ", fileName, " :"
    plt.imshow(gray_image)
    plt.show()
    gray_image = np.reshape(gray_image, (conf.PICTURE_DIM_EMOTION, conf.PICTURE_DIM_EMOTION, 1))

    # emotion prediction
    emotion_prediction = emotions_model.predict(np.array([gray_image]), batch_size = 1)[0]
    
    print 
    print emotion_prediction
    print
    print "predicted emotion:", conf.emotion.items()[np.argmax(emotion_prediction)][1], "with prob:", np.max(emotion_prediction)
    print
    
    # If pain emotion predicted, predict its level
    if np.argmax(emotion_prediction) == 7 :
        
        painLevel_prediction = pain_model_adam.predict(np.array([image]), batch_size = 1)[0]
        
        print "predicted pain level:", np.argmax(painLevel_prediction)

#### - Prediction of Pain level

In [ ]:
for f in os.listdir(test_pain_imgs_path):
    fileName = os.path.join(test_pain_imgs_path, f)
    image = imgToColoredMatrix(fileName)
    painLevel_prediction = pain_model_adam.predict(np.array([image]), batch_size = 1)[0]
    
    print 
    print painLevel_prediction
    print
    print "predicted pain level:", np.argmax(painLevel_prediction), "with prob:", np.max(painLevel_prediction)
    